In [32]:
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [33]:
# %mkdir -p modelsave checkpoint/
# %pip install --force-reinstall /home/z1933/workplace/machinelearning/PII-Data-Detection/unidecode/Unidecode-1.3.8-py3-none-any.whl

In [34]:
from cfg import *
from util import *
Config.modelsavepath

'/home/z1933/workplace/machinelearning/PII-Data-Detection//modelsave/'

In [35]:
##### main
data = json.load(open(Config.data_path+"/train.json"))

# downsampling of negative examples
p=[] # positive samples (contain relevant labels)
n=[] # negative samples (presumably contain entities that are possibly wrongly classified as entity)
for d in data:
    if any(np.array(d["labels"]) != "O"): p.append(d)
    else: n.append(d)
print("original datapoints: ", len(data))

external = json.load(open(Config.data_path+"/mixtral-8x7b-v1.json"))
print("external datapoints: ", len(external))

moredata = json.load(open(Config.data_path+"/moredata_dataset_fixed.json"))
print("moredata datapoints: ", len(moredata))

data = moredata+external+p+n[:len(n)//3]
print("combined: ", len(data))


original datapoints:  6807
external datapoints:  2355
moredata datapoints:  2000
combined:  7254


In [36]:
data_pd=pd.DataFrame(data)
len(data_pd[data_pd['tokens'].apply(len)>2024])

14

In [37]:
tokenizer = AutoTokenizer.from_pretrained(Config.model_name)
tokenized_input =tokenizer(data_pd.iloc[data_pd.full_text.str.len().idxmax()]['full_text'],return_tensors='pt')

/home/z1933/miniconda3/envs/d2l/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [38]:
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokenized_input["input_ids"].size()

torch.Size([1, 2831])

In [39]:
from itertools import chain
#将所有数据的labels连接在一起,然后查重,转成list的格式,然后从小到大排序
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
print(f"all_labels:{all_labels}")
label2id = {l: i for i,l in enumerate(all_labels)}
#这个是{id:label},上面是{label:id}
id2label = {v:k for k,v in label2id.items()}

# Open a file for writing
with open(Config.modelsavepath+"//idlabel.json", "w") as f:
    # Write the map to the file in JSON format
    json.dump({'label2id':label2id,'id2label':id2label}, f)
id2label

all_labels:['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O']


{0: 'B-EMAIL',
 1: 'B-ID_NUM',
 2: 'B-NAME_STUDENT',
 3: 'B-PHONE_NUM',
 4: 'B-STREET_ADDRESS',
 5: 'B-URL_PERSONAL',
 6: 'B-USERNAME',
 7: 'I-ID_NUM',
 8: 'I-NAME_STUDENT',
 9: 'I-PHONE_NUM',
 10: 'I-STREET_ADDRESS',
 11: 'I-URL_PERSONAL',
 12: 'O'}

In [40]:
# EDA
tranjs_path="dataset/train.json"
train_df = json.load(open(tranjs_path))
print(f"len(train_df):{len(train_df)}, train_df[0].keys(): {list(train_df[0].keys())}")
print("-"*50)


print(f"train_df length:", len(train_df))

labels = set()
label_counts = {}
for i in range(len(train_df)):
    labels.update(train_df[i]['labels'])
    for label in train_df[i]['labels']:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
            
print(f"labels: {labels}")
print('-'*25)
print(f"label_counts: {label_counts}")

len(train_df):6807, train_df[0].keys(): ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels']
--------------------------------------------------
train_df length: 6807
labels: {'B-ID_NUM', 'I-STREET_ADDRESS', 'B-STREET_ADDRESS', 'I-PHONE_NUM', 'I-ID_NUM', 'O', 'I-NAME_STUDENT', 'I-URL_PERSONAL', 'B-PHONE_NUM', 'B-URL_PERSONAL', 'B-NAME_STUDENT', 'B-USERNAME', 'B-EMAIL'}
-------------------------
label_counts: {'O': 4989794, 'B-NAME_STUDENT': 1365, 'I-NAME_STUDENT': 1096, 'B-URL_PERSONAL': 110, 'B-EMAIL': 39, 'B-ID_NUM': 78, 'I-URL_PERSONAL': 1, 'B-USERNAME': 6, 'B-PHONE_NUM': 6, 'I-PHONE_NUM': 15, 'B-STREET_ADDRESS': 2, 'I-STREET_ADDRESS': 20, 'I-ID_NUM': 1}


In [41]:

# # EDA
# # display bar
# import copy
# display_label=copy.copy(label_counts)
# del  display_label['O']
# y =  [i for i in display_label.values()]

# g=sns.barplot(x=list(display_label.keys()),y=y)
# g.set_xticklabels(g.get_xticklabels(), rotation=90)

In [42]:
tmp_list=[]
for i in train_df:
    for j in range(0,len(i['tokens'])):
        if i['tokens'][j].isspace() and i['labels'][j]!='O':
            tmp_list.append([i['tokens'][j-1],i['labels'][j-1],i['tokens'][j],i['labels'][j],i['tokens'][j+1],i['labels'][j+1]])
tmp_list

[['656',
  'I-STREET_ADDRESS',
  '\n',
  'I-STREET_ADDRESS',
  'Joshuamouth',
  'I-STREET_ADDRESS'],
 ['419',
  'I-STREET_ADDRESS',
  '\n',
  'I-STREET_ADDRESS',
  'Andreahaven',
  'I-STREET_ADDRESS']]

In [43]:
df_train=pd.DataFrame(train_df)
df_train_tokens=[]
for row in df_train.itertuples(index=False):
    for i in range(len(row.tokens)):
        df_train_tokens.append((row.document,row.tokens[i],row.labels[i],row.trailing_whitespace[i]))
df_train_tokens=pd.DataFrame(df_train_tokens,columns=['document','tokens','labels','trailing_whitespace'])


In [44]:
df_train_tokens.loc[df_train_tokens['tokens']=='\n']

,document,tokens,labels,trailing_whitespace
861205,9854,\n,I-STREET_ADDRESS,False
1445336,11442,\n,I-STREET_ADDRESS,False


In [45]:
df_train_tokens.iloc[861206]

document                           9854
tokens                      Joshuamouth
labels                 I-STREET_ADDRESS
trailing_whitespace               False
Name: 861206, dtype: object

In [46]:
# inspect specific label
inspect="PHONE"
df_temp=df_train_tokens.loc[ df_train_tokens.labels.str.contains(inspect)]
print(df_temp.to_string())

        document        tokens       labels  trailing_whitespace
150909      4381             (  B-PHONE_NUM                False
150910      4381       320)202  I-PHONE_NUM                False
150911      4381             -  I-PHONE_NUM                False
150912      4381    0688x95843  I-PHONE_NUM                 True
179880      4777             (  B-PHONE_NUM                False
179881      4777       223)392  I-PHONE_NUM                False
179882      4777             -  I-PHONE_NUM                False
179883      4777          2765  I-PHONE_NUM                False
286835      6243             (  B-PHONE_NUM                False
286836      6243       820)913  I-PHONE_NUM                False
286837      6243             -  I-PHONE_NUM                False
286838      6243      3241x894  I-PHONE_NUM                False
287266      6243             (  B-PHONE_NUM                False
287267      6243       820)913  I-PHONE_NUM                False
287268      6243         

In [47]:
input_data=pd.DataFrame(data)

In [48]:
# DEA
# print all unicode character
# def has_unicode_codepoints(text):
#     return any(ord(char) > 127 for char in text)
# tmp_set=set()
# pattern =re.compile(r"[^\u0000-\u00ff]")
# for i in data:
#     for j in i['tokens']:
#         if has_unicode_codepoints(j):
#             tmp_set.add(j)
# for i in tmp_set:
#     print(i)

In [49]:
# !cp ./nltk_data/* ~/nltk_data/ -r

In [50]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "labels": [x["labels"] for x in data]
})


预测时，空缺值填'O'

In [51]:
# EDA
# import matplotlib.pyplot as plt
# import seaborn as sns
# tmp_list=[]
# for doc in data:
#     full_text=doc['full_text']
#     split_lines=full_text.split('\n\n')
#     for i in split_lines:
#         tmp_list.append(len(tokenizer(i)['input_ids']))
# print(len(tmp_list))
# print(max(tmp_list))
# sns.displot(tmp_list)

In [52]:
# 根据'\n\n'分token
preprocesssed_ds=ds.map(train_preprocesss, fn_kwargs={'tokenizer':tokenizer,'label2id':label2id},num_proc=10,desc="prepocessing data")
preprocesssed_ds

prepocessing data (num_proc=10):   0%|          | 0/7254 [00:00<?, ? examples/s]

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'labels', 'bertlabels', 'berttokenids', 'berttokenmask', 'berttokentoken_type_ids'],
    num_rows: 7254
})

In [53]:
tmp_str="."
tmp_res=tokenizer(tmp_str)
tmp_res

{'input_ids': [1, 323, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [54]:
tokenizer.convert_ids_to_tokens(tmp_res['input_ids'])

['[CLS]', '▁.', '[SEP]']

In [55]:
# rebuid dataset 
tmp_pd=expanddataset(preprocesssed_ds)
print(tmp_pd['berttokenids'].str.len().agg(['mean','max','std','min']))
tmp_pd.loc[tmp_pd['berttokenids'].str.len()==3]

mean     52.748827
max     933.000000
std      56.812397
min       3.000000
Name: berttokenids, dtype: float64


,full_text,document,tokens,trailing_whitespace,labels,berttokenids,berttokenmask,berttokentoken_type_ids,bertlabels
15384,Charles-Edouard Siegel | c-siegel@outlook.com;...,wcyzegeled,"[Charles, -, Edouard, Siegel, |, c-siegel@outl...","[False, False, True, True, True, False, True, ...","[B-NAME_STUDENT, I-NAME_STUDENT, I-NAME_STUDEN...","[1, 5736, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
16630,Domenico Childers - 26973 Jennifer Ford\nJimmy...,kvyrxtluau,"[Domenico, Childers, -, 26973, Jennifer, Ford,...","[True, True, True, True, True, False, False, F...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-STREET_A...","[1, 10766, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
17711,"Johanny Arnold | PSC 3616, Box 9945\nAPO AE 52...",cyherfmagt,"[Johanny, Arnold, |, PSC, 3616, ,, Box, 9945, ...","[True, True, True, True, False, True, True, Fa...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-STREET_A...","[1, 897, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
17903,Yadiel Pate; yadiel_p@mail.com; ROLL NO 307955...,jtwydkjjiw,"[Yadiel, Pate, ;, yadiel_p@mail.com, ;, ROLL, ...","[True, False, True, False, True, True, True, F...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-EMAIL, O...","[1, 1124, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
21182,Anna Saleh-Parish; 59813 Weaver Neck Suite 623...,taxrdujluv,"[Anna, Saleh, -, Parish, ;, 59813, Weaver, Nec...","[True, False, False, False, True, True, True, ...","[B-NAME_STUDENT, I-NAME_STUDENT, I-NAME_STUDEN...","[1, 577, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
...,...,...,...,...,...,...,...,...,...
101379,Challenge\n\nThe challenge is to introduce the...,13539,"[Challenge, \n\n, The, challenge, is, to, intr...","[False, False, True, True, True, True, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1, 6738, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
101400,Example Reflection – Mind Mapping\n\nChallenge...,13547,"[Example, Reflection, –, Mind, Mapping, \n\n, ...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1, 5736, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
101411,Example Reflection – Mind Mapping\n\nChallenge...,13547,"[Example, Reflection, –, Mind, Mapping, \n\n, ...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1, 22459, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"
101417,Example Reflection – Mind Mapping\n\nChallenge...,13547,"[Example, Reflection, –, Mind, Mapping, \n\n, ...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1, 17798, 2]","[1, 1, 1]","[0, 0, 0]","[12, 12, 12]"


In [56]:
full_ds=Dataset.from_pandas(tmp_pd)
print(full_ds)

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'labels', 'berttokenids', 'berttokenmask', 'berttokentoken_type_ids', 'bertlabels'],
    num_rows: 101484
})


In [57]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
from transformers import DataCollatorForTokenClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_len=int(0.9*len(full_ds)) # 长度分割，兼容老版本torch
train_dataset, val_dataset = random_split(full_ds, [train_len,len(full_ds)-train_len],generator=torch.Generator().manual_seed(Config.seed)) #TODO:改变比例


In [58]:
print(f"train dataset len:{len(train_dataset)}")
print(f"vail dataset len:{len(val_dataset)}")

train dataset len:91335
vail dataset len:10149


In [59]:
data_collator = Collate(tokenizer=tokenizer)
train_dataloader=DataLoader(train_dataset,batch_size=Config.batch_size,pin_memory=True,collate_fn=data_collator,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=Config.batch_size,pin_memory=True,collate_fn=data_collator)

In [60]:
from model1 import NLPModel
model=NLPModel(id2label,label2id,Config.model_name).to(device)
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
import gc

optimizer= AdamW(model.parameters(),lr=Config.lr,weight_decay=Config.weight_decay)

scaler = torch.cuda.amp.GradScaler()
valstep = len(train_dataloader) //Config.evaltimes

num_train_steps = int(len(train_dataloader) * Config.epochs)
scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=Config.num_warmup_steps, num_training_steps=num_train_steps, 
            )

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
def compute_metrics(predictions,traget, all_labels):
    
    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, traget)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, traget)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

def validation(val_dataloader,model):
    model.eval()

    allprecitions=[]
    alltargets=[]
    for step,dataset in enumerate(tqdm(val_dataloader)):

        with torch.no_grad():
            ids = dataset["ids"].to(device,non_blocking=True)
            mask = dataset["mask"].to(device,non_blocking=True)
            targets = dataset["targets"].to(device,non_blocking=True)
            tokentype = dataset["type_ids"].to(device,non_blocking=True)
            logit,loss = model(ids,mask, tokentype,targets)

            targets=targets.view(-1)
            targets = targets.detach().cpu().numpy()

            allprecitions.append(np.argmax(logit, axis=1))
            alltargets.append(targets)
        del ids,mask,targets,tokentype,loss,logit

    score=compute_metrics(allprecitions,alltargets,all_labels)
    
    gc.collect()
    torch.cuda.empty_cache()

    model.train()

    return score

In [31]:
gc.collect()
torch.cuda.empty_cache()
start_time=time.time()
bestscore=0
epoch=0

if Config.resume_train_epoch:
    checkpoint = torch.load(Config.workdir+f"//checkpoint//checkpoint{Config.resume_train_epoch}.pth")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    epoch = checkpoint['epoch']
    bestscore=checkpoint['bestscore']
    model.train()
    print(f"start epoch:{epoch}")

while epoch <Config.epochs:
    losses=[] 
    for step,dataset in enumerate(tqdm(train_dataloader)):
        with torch.cuda.amp.autocast():
            ids = dataset["ids"].to(device,non_blocking=True)
            mask = dataset["mask"].to(device,non_blocking=True)
            tokentype = dataset["type_ids"].to(device,non_blocking=True)
            targets = dataset["targets"].to(device,non_blocking=True)

            logit,loss = model(ids,mask, tokentype,targets)
            loss/= Config.accumulation_steps

        # Accumulates scaled gradients.
        scaler.scale(loss).backward()
        # losses.append(loss.item())
        
        if (step+1) % Config.accumulation_steps ==0:
            scaler.step(optimizer)
            scale = scaler.get_scale()
            # Updates the scale for next iteration.
            scaler.update()
            skip_lr_sched = (scale != scaler.get_scale())
            if not skip_lr_sched:
                scheduler.step()
            optimizer.zero_grad()

        # validation
        if epoch>0 and (step+1) % valstep == 0:
            scores=validation(val_dataloader,model)
            print("Validation:Epoch{} Step [{}/{}] Loss: {:.3f} Time: {:.1f} Metric: {}".format(epoch,step, len(train_dataloader)-1, loss.item(), time.time()-start_time,scores))
            if scores['f1']>bestscore:
                print(f"Best score is {bestscore} → {scores['f1']}. Saving model")
                torch.save(model.state_dict(), os.path.join(Config.modelsavepath,"model_seed{}_score{:.3f}.pth".format(Config.seed,bestscore)))
                bestscore=scores['f1']
            else:
                print("no improvement, bestf1 is {:.3f} score is {}".format(bestscore,scores))
                # report loss
        if (step+1) % Config.logging_steps ==0:
            # print("Logging: Step [{}/{}] Loss: {:.3f} Time: {:.1f}".format(step, len(train_dataloader)-1, loss.item(), time.time()-start_time))
            gc.collect()
            torch.cuda.empty_cache()
        del ids,mask,targets,tokentype,loss,logit

    epoch+=1
    # checkpoint
    torch.save({
            'epoch': epoch,
            'bestscore':bestscore,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict':scheduler.state_dict(),
            'scaler_state_dict':scaler.state_dict()
            }, Config.modelsavepath+"/checkpoint{}.pth".format(epoch))

tokenizer.save_pretrained(Config.modelsavepath)
os.system("/usr/bin/shutdown")

  0%|          | 4/39539 [00:24<67:33:22,  6.15s/it] 


KeyboardInterrupt: 